# Desafio

Executar o fine-tuning de um foudation model (Llama, Bert, Minstrel, etc), utilizando o dataset "TheAmazonTtiles-1.3MM". <br>
O modelo treinado deverá:
- Receber perguntas com um contexto obtido por meio de uma integração RAG (Retrieve-and-Generate), utilizando documentos relacionados aos produtos da Amazon.
- A partir do prompt formado pela pergunta do usuário e dos dados retornados do RAG, o modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do RAG, incluindo as fontes.

# Passos para a aplicação de fine-tuning no Modelo

O The AmazonTitles-1.3MM consiste em consultas textuais reais de usuários e títulos associados de produtos relevantes encontrados na Amazon, medidos por ações implícitas ou explícitas dos usuários.
1.   Preparação do Dataset <br>
    - Download do dataset AmazonTitles-1.3MM.
    - Prepare os dados para o fine-tuning, garantindo que estejam organizados de maneira adequada para o treinamento do modelo.
    - Limpe e pré-processe os dados conforme necessário para o modelo escolhido.
2.  Execução do Fine-Tuning <br>
    - Execute o fine-tuning do foundation model selecionado utilizando o dataset preparado.
    - Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.
3.  Configuração da Integração RAG
    - Configure uma integração RAG (Retrieve-and-Generate) para fornecer contexto ao modelo a partir dos documentos relacionados aos produtos da Amazon.
    - Certifique-se de que a integração esteja funcionando corretamente para recuperar e fornecer dados contextuais ao modelo.
4.  Geração de Respostas
    - Configure o modelo treinado para receber perguntas dos usuários.
    - Quando uma pergunta for recebida, utilize a integração RAG para recuperar informações relevantes do dataset AmazonTitles-1.3MM.
    - Combine a pergunta do usuário e os dados retornados do RAG para formar um prompt completo.
    - O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do RAG, incluindo as fontes fornecidas.

# Preparação do Dataset
- Download do dataset AmazonTitles-1.3MM.
- Prepare os dados para o fine-tuning, garantindo que estejam organizados de maneira adequada para o treinamento do modelo.
- Limpe e pré-processe os dados conforme necessário para o modelo escolhido.

In [1]:
FOLDER_PATH = '/content/drive/MyDrive/TheAmazonTitles'
FINE_TUNING_PATH = f'{FOLDER_PATH}/fine-tuning'
URLDATA_GOOGLEDRIVE = f'https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK'

In [2]:
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dado que os dados disponibilizados para construção do modelo encontram-se em um link do Google Drive, se faz necessário obter alguma estratégia para manuseio.
No modelo, utilizando a lib gdown, é feito o download a partir do link e os arquivos extraídos para uma pasta em MyDrive.
Se faz necessária essa estratégia pois garante que, ao encerrar a sessão, os dados não são perdidos, gerando a necessidade de realizar o download e extração novamente.

In [3]:
import gdown
import zipfile
import os

if not os.path.exists(FOLDER_PATH):
    os.makedirs(FOLDER_PATH)

zip_file = f'{FOLDER_PATH}/LF-Amazon-1.3M.zip'

gdown.download(URLDATA_GOOGLEDRIVE, zip_file, quiet=False)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(FOLDER_PATH)

print("Files successfully extracted and saved to the Google Drive directory!")

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=0659c183-dab2-4dcc-b2dd-051d78236a4a
To: /content/drive/MyDrive/TheAmazonTitles/LF-Amazon-1.3M.zip
100%|██████████| 890M/890M [00:43<00:00, 20.5MB/s]


Files successfully extracted and saved to the Google Drive directory!


Estrutura do Dowload de AmazonTitles-1.3MM é a seguinte:

- pasta.zip e a mesma abriga os seguintes arquivos:
    - filter_labels_train.txt (padrão do conteúdo do arquivo: número número)
    - filter_labels_test.txt (padrão do conteúdo do arquivo: número número)
    - tst.json.gz (padrão do conteúdo do arquivo: objeto json inline)
    - trn.json.gz (padrão do conteúdo do arquivo: objeto json inline)
    - lbl.json.gz (padrão do conteúdo do arquivo: objeto json inline)

Todos os arquivos acima passam das 100 mil linhas cada um. <br>
Para lidar com arquivos grandes como os .json.gz, carregar todo o conteúdo na memória pode ser inviável por falta de recursos computacionais. <br>
A abordagem adotada foi aplicar o algoritmo de Reservoir Sampling: permite selecionar uma amostra aleatória de tamanho fixo de um fluxo de dados de tamanho desconhecido sem carregar todo o conteúdo do arquivo em memória.

In [4]:
import gzip
import random
import json

def reservoir_sampling_gz_json_with_indices(file_path, sample_size):
    sample = []
    indices = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line)
                if i < sample_size:
                    sample.append(obj)
                    indices.append(i)
                else:
                    j = random.randint(0, i)
                    if j < sample_size:
                        sample[j] = obj
                        indices[j] = i
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {i}: {e}")
    return sample, indices


In [5]:
# Tamanho das amostras desejadas
sample_size_train = 4000
sample_size_test = 1000
sample_size_label = 4000

# Realiza a amostragem dos arquivos gzip de treinamento e teste
train_sample, train_indices = reservoir_sampling_gz_json_with_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/trn.json.gz', sample_size_train)
test_sample, test_indices = reservoir_sampling_gz_json_with_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/tst.json.gz', sample_size_test)

print(len(train_sample))
print(len(test_sample))

4000
1000


Após a extração dos dados de treino e teste, é o momento de extrair o conteúdo dos arquivos de filter_labels para indexar com os dados extraídos anteriormente. E assim criar um mapeamento dos dados com base nos filtros.

In [6]:
# Carregar os labels correspondentes aos índices
def load_labels_by_indices(file_path, indices):
    labels = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
    for idx in indices:
        if idx < len(lines):
            labels.append(lines[idx].strip().split())
        else:
            labels.append([])
    return labels

filter_labels_train_sample = load_labels_by_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/filter_labels_train.txt', train_indices)
filter_labels_test_sample = load_labels_by_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/filter_labels_test.txt', test_indices)

print(len(filter_labels_train_sample))
print(filter_labels_train_sample[:100])

4000
[[], [], [], [], [], ['1176652', '736535'], [], ['263600', '445493'], [], [], [], [], [], [], [], [], [], [], ['10324', '41505'], ['395171', '351456'], [], [], [], [], [], ['805206', '712900'], [], ['820313', '687670'], [], [], ['324437', '205387'], [], [], ['503218', '223410'], [], [], [], [], [], ['1505094', '454723'], [], [], [], ['268242', '310523'], [], [], [], [], ['581112', '566235'], [], ['1151487', '240581'], [], [], [], [], ['29579', '14897'], [], [], [], ['1487196', '295488'], [], [], [], [], [], [], ['1834344', '1282492'], ['92548', '309315'], [], ['10643', '67692'], [], ['1799307', '383888'], [], [], [], ['370244', '55135'], ['1945255', '837203'], ['45307', '197579'], ['1455401', '480423'], [], [], ['509003', '337154'], ['807336', '841473'], ['1126364', '1145116'], [], [], ['1227244', '1039860'], ['1685034', '687219'], ['493712', '119049'], [], [], ['1049443', '882525'], [], [], ['2054566', '433307'], [], ['1317406', '393311'], [], [], ['1302325', '1042213']]


## Realizando a limpeza dos dados

É de extrema importância a etapa de limpeza, pois é a partir dela que geramos inputs sem ruídos para o treinamento do modelo.
Abaixo, foi implementada limpeza de HTML Entities nos campos de texto, além de outros tratamentos necessários como remoção de novas linhas e múltiplos espaços.

In [9]:
import re
import html

def unescape_html_entities(text):
    text = html.unescape(text)
    def replace_hex_entity(match):
        code_point = int(match.group(1), 16)
        return chr(code_point)

    text = re.sub(r'&#x([0-9A-Fa-f]+);', replace_hex_entity, text)
    return text

def clean_text(text):
    text = unescape_html_entities(text)
    text = text.replace('’', "'")
    # Remove caracteres especiais
    text = re.sub(r"[^A-Za-z0-9',. ]+", '', text)
    # Remove novas linhas e espaços extras
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    # Remove múltiplos espaços
    text = re.sub(' +', ' ', text)
    return text


In [10]:
from typing import List, Dict

# Processa uma lista de amostras, limpando textos e associando rótulos. Remove itens que não possuem 'title' ou 'uid'.
def process_samples(sample_data: List[Dict], filter_labels_sample: List) -> List[Dict]:
    processed_samples = []
    total_items = len(sample_data)
    total_labels = len(filter_labels_sample)

    for idx, item in enumerate(sample_data):
        title = item.get('title', '').strip()
        uid = item.get('uid', '').strip()
        content = item.get('content', '').strip()

        if not title or not uid or not content:
            continue  # Ignora itens com 'title' ou 'uid' vazios ou 'content' vazios

        if title == content:
            continue  # Ignora itens com 'title' == 'content'

        # Limpeza dos campos de texto
        item['title'] = clean_text(title)
        item['content'] = clean_text(content)

        # Associação de rótulos
        if idx < len(filter_labels_sample):
            item['labels'] = filter_labels_sample[idx]
        else:
            item['labels'] = []

        processed_samples.append(item)

    return processed_samples

In [11]:
clean_train_sample = process_samples(train_sample, filter_labels_train_sample)
clean_test_sample = process_samples(test_sample, filter_labels_test_sample)

print(clean_train_sample[0])
print(clean_test_sample[0])

{'uid': 'B003JOVL70', 'title': 'Instructor Size Underwater Writing Dive Slates 8 x 10, 8 x 11 Slate SL04', 'content': 'ideal for note taking or a back up communication device.', 'target_ind': [150234, 600808], 'target_rel': [1.0, 1.0], 'labels': []}
{'uid': 'B007Y9NRI2', 'title': 'Aimo Wireless LGLM272PCDI123 Bling Brilliance Premium Grade Diamond Case for LG Rumor ReflexFreedomConverseExpression C395 Retail Packaging Pink Leopard', 'content': 'The Pink Leopard Premium Executive Diamond Case by Aimo Wireless for your LG Rumor ReflexFreedomConverseExpression C395 is a durable slim fit SnapOn case made to add luxury, style and protection to your phone. The case is constructed of durable hard plastic adorned with beautiful rhinestone crystals to give it a unique, oneofakind design and protection to match. The Pink Leopard Executive diamond design really gives the cover brilliance and shine, complementing the look and feel of your LG Rumor ReflexFreedomConverseExpression C395. Accurate cut

# Execução do Fine-Tuning
- Execute o fine-tuning do foundation model selecionado utilizando o dataset preparado.
- Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.

In [7]:
!pip install datasets
!pip install transformers accelerate

Para esse modelo foi escolhido o distilgpt2 por ser uma versão menor e mais leve do GPT-2, projetada para ser mais eficiente em termos de recursos. Como o modelo é para fins acadêmicos, é o suficiente para aplicar as lógicas necessárias.
E para treinar esse modelo, são necessárias algumas etapas prévias:

1. Preparação dos Dados de Treinamento, processando amostras de dados limpos para criar pares de "prompt" e "completion".
2. Criação de um Dataset, convertendo essas amostras em um formato adequado para treinamento utilizando a biblioteca datasets.

In [12]:
from datasets import Dataset

train_data = clean_train_sample
test_data = clean_test_sample

# Prepara os dados para o dataset
def get_training_samples(sample):
    training_samples = []
    for item in sample:
        # Integração das labels no prompt
        prompt = f"Category: {item['labels']}\nTitle: {item['title']}\nDescription:"
        completion = item['content']
        training_samples.append({'prompt': prompt, 'completion': completion})
    return training_samples

# Cria Dataset
train_dataset = Dataset.from_list(get_training_samples(train_data))
test_dataset = Dataset.from_list(get_training_samples(test_data))

In [13]:
train_dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 2494
})


3. Configuração e Carregamento do Modelo e Tokenizer, através do modelo pré-treinado distilgpt2 da biblioteca transformers. Ajusta o tokenizer e o modelo para garantir que estejam alinhados em termos de tokens.
4. Tokenização dos Dados, que transforma os textos de entrada em tokens numéricos que o modelo pode processar. Em linhas gerais, prepara os dados para serem alimentados no modelo durante o treinamento.

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'distilgpt2'

# Carrega o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
'''
Faz a atribuição do pad_token.
O GPT-2 originalmente não possui um token de padding,
e essa substituição ajuda na tokenização de sequências de comprimento fixo.
'''
tokenizer.pad_token = tokenizer.eos_token

# Carrega o modelo escolhido
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))  # Redimensiona o Token Embeddings

# Função de tokenização
def tokenize_function(examples):
    # Concatena 'prompt' e 'completion' para cada item
    inputs = [prompt + " " + completion for prompt, completion in zip(examples['prompt'], examples['completion'])]
    # Tokeniza os inputs
    tokenized_inputs = tokenizer(
        inputs,
        truncation=True,       # Garante que sequências mais longas que max_length sejam truncadas.
        padding='max_length',  # Adiciona padding para que todas as sequências tenham o mesmo comprimento
        max_length=128,        # Define o comprimento máximo das sequências tokenizadas.
    )
    return tokenized_inputs


tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2494 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset

Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 2494
})

In [16]:
from transformers import DataCollatorForLanguageModeling

'''
Configura um objeto responsável por preparar os dados em lotes para o
treinamento do modelo de linguagem.
'''
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Masked Language Modeling = False / Causal Language Modeling = True.
)


In [17]:
from transformers import TrainingArguments, Trainer

# Configuração dos argumentos para Treinamento
training_args = TrainingArguments(
    output_dir=f'{FINE_TUNING_PATH}/training_args',
    num_train_epochs=3,  # Número de vezes que o conjunto de dados completo será passado pelo modelo durante o treinamento.
    per_device_train_batch_size=4, # Tamanho do lote por dispositivo (GPU/CPU) durante o treinamento.
    save_total_limit=2, # Número máximo de checkpoints a serem mantidos. Os mais antigos são removidos quando o limite é excedido.
    logging_steps=100, # Intervalo de passos entre cada registro (log) de métricas de treinamento.
    learning_rate=5e-5          # Ajustar a taxa de aprendizado
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Inicializando treino dos dados com o modelo distilgpt2
trainer.train()

Step,Training Loss
100,4.369200
200,4.283800
300,4.269300
400,4.184300
500,4.185500
600,4.191600
700,3.958500
800,3.901200
900,3.875500
1000,3.959000


TrainOutput(global_step=1872, training_loss=3.955227110120985, metrics={'train_runtime': 244.2112, 'train_samples_per_second': 30.637, 'train_steps_per_second': 7.665, 'total_flos': 244377785991168.0, 'train_loss': 3.955227110120985, 'epoch': 3.0})

A tendência geral de diminuição da perda indica que o modelo está aprendendo e melhorando seu desempenho ao longo do treinamento.
A estabilização da perda sugere que o modelo está alcançando um ponto de convergência.

In [18]:
# Salvando componentes do Treino após a execução para não retreinar a cada queda de sessão
trainer.save_model(f'{FINE_TUNING_PATH}/trainer-result')
model.save_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')
tokenizer.save_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')

('/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/vocab.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/merges.txt',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/tokenizer.json')

# Configuração da Integração RAG
- Configure uma integração RAG (Retrieve-and-Generate) para fornecer contexto ao modelo a partir dos documentos relacionados aos produtos da Amazon.
- Certifique-se de que a integração esteja funcionando corretamente para recuperar e fornecer dados contextuais ao modelo.


In [19]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence_transformers faiss-cpu

In [22]:
import pandas as pd

clean_train_sample_df = pd.DataFrame(clean_train_sample)
clean_train_sample_df.reset_index(drop=True, inplace=True)
clean_train_sample_df.head()

,uid,title,content,target_ind,target_rel,labels
0,B003JOVL70,Instructor Size Underwater Writing Dive Slates...,ideal for note taking or a back up communicati...,"[150234, 600808]","[1.0, 1.0]",[]
1,B003KRRAAI,Popup Towel Mini Wipes Bag of 10,PopUp Towels are perfect for any use. These al...,[457842],[1.0],[]
2,B000QV0MA6,"Gumballs Sugar Free 16oz,",Carousel Sugar Free Gumballs .62Carousel sugar...,"[12384, 46466, 46468, 51979, 55729, 106099, 11...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
3,1484843010,Hard Math for Elementary School Workbook,Glenn Ellison is the Gregory K. Palm Professor...,"[7512, 14785, 18541, 18545, 18620, 18657, 1867...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1176652, 736535]"
4,0415420830,Paradoxes from A to Z,'Selfcontained courses in paradox are not usua...,[72776],[1.0],"[263600, 445493]"


## Conversão DataFrame para Document do LangChain

Organizar os dados em objetos Document facilita o gerenciamento e a manipulação dos documentos, especialmente quando se trata de adicionar metadados que podem ser úteis para buscas e filtragens futuras.
Além de que a estruturação dos documentos de forma consistente permite que os embeddings sejam gerados de maneira uniforme, garantindo que todas as informações relevantes (como título e UID) estejam disponíveis para uso posterior.

In [23]:
import pandas as pd
from langchain.docstore.document import Document
from tqdm import tqdm

def create_documents(df):
    """
    Converte as entradas do DataFrame em objetos Document do LangChain
    """
    documents = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Criando Objetos Documentos"):
        content = row['content']
        metadata = {'uid': row['uid'], 'title': row['title']}
        documents.append(Document(page_content=content, metadata=metadata))
    return documents

documents = create_documents(clean_train_sample_df)
print('\n',documents[0])

Criando Objetos Documentos: 100%|██████████| 2494/2494 [00:01<00:00, 1954.64it/s]


 page_content='ideal for note taking or a back up communication device.' metadata={'uid': 'B003JOVL70', 'title': 'Instructor Size Underwater Writing Dive Slates 8 x 10, 8 x 11 Slate SL04'}


## Criação de Embeddings e Retriever
Embeddings transformam texto em vetores numéricos que capturam semântica e contexto, permitindo que algoritmos de busca e recuperação encontrem similaridades entre diferentes textos de forma eficiente. E a lib FAISS permite indexar e buscar rapidamente vetores de alta dimensão, tornando-o ideal para aplicações que exigem buscas por similaridade em grandes conjuntos de dados, como sistemas de recomendação, busca semântica e recuperação de informações.

`all-MiniLM-L6-v2` foi escolhido por ser conhecido pela eficiência em produzir embeddings de alta qualidade para tarefas de processamento de linguagem natural.




In [24]:
embedding_model_name="all-MiniLM-L6-v2"

In [25]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

def load_faiss_retriever():
    """
    Carrega o índice FAISS e configura o retriever
    """
    embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Recupera 5 documentos
    return retriever

retriever = load_faiss_retriever()


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-25-618e0abc1797>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)


Verificando se o retriver está hábil a consultar Documentos contextualizados com o input fornecido.

In [26]:
docs = retriever.get_relevant_documents("Tell me something about Marvel Movies")[:10]
docs

<ipython-input-26-bff30c2fe6ac>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Tell me something about Marvel Movies")[:10]


[Document(metadata={'uid': 'B004NE4KX2', 'title': 'Marvel Superhero Squad Series 20 Mini 3 Inch Figure 2Pack Iron Man Red Hulk'}, page_content='Spring into adventure with the worlds greatest collection of Super Heroes. The mightiest Marvel heroes are ready to save the day with their amazing abilities and super powers. Build your team and join in the battle with the Marvel Super Hero Squad Gear up for adventure with this fun duo Pair of chunky figures includes SpiderMan and Moon Knight figures.'),
 Document(metadata={'uid': 'B00DDVY6C6', 'title': 'Gentle Giant SpiderMan Mini Bust, RedBlue'}, page_content="Few heroes are as synonymous with Marvel than SpiderMan. For over 50 years now, the WallCrawler has dispensed his unique form of wisecracking, webbed justice on the streets of New York and throughout the Marvel Universe. Although he has worn many different uniforms over the years, the classic redandblue suit will always be the first thing fans imagine when they think of SpiderMan. Now,

## Reutilização de Modelos Fine-Tuned
Utiliza o modelo e tokenizer treinados previamente com o model `distilgpt2`. Permitindo o aproveitamento do conhecimento adquirido durante o fine-tuning, garantindo que o modelo esteja adaptado às necessidades específicas.

In [27]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import torch
import re

model_path=f'{FINE_TUNING_PATH}/model-tokenizer'

def load_finetuned_model():
    """
    Carrega o modelo GPT-2 ajustado e o tokenizer
    """
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token  # Assegura que o token de padding esteja definido
    model = GPT2LMHeadModel.from_pretrained(model_path)

    # Move o modelo para GPU se disponível
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return model, tokenizer, device

# Inicializa o modelo, o tokenizer e o dispositivo
model, tokenizer, device = load_finetuned_model()

In [28]:
# Inicializa o pipeline de geração de texto
nlp = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Usa GPU se disponível
)

## Personalizando as Respostas com o RAG

E abaixo, os métodos responsáveis pela captação dos documentos e geração das respostas das questões inputadas pelo usuário.

Com esses métodos é possível:
1. Fornecer ao modelo um contexto relevante e conciso, aumentando a probabilidade de gerar respostas precisas e informativas.
2. Truncar o contexto garante que o prompt completo (incluindo a pergunta e o contexto) não exceda o limite.
3. Retornar as fontes permite que você saiba de onde as informações foram extraídas, o que é útil para verificação e transparência.

In [45]:
max_documents=10
max_context_tokens=500
max_new_tokens=128


def get_relevant_context(question):
    """
    Recupera o contexto relevante para a pergunta dada usando o retriever.
    """
    # Recupera documentos relevantes
    docs = retriever.get_relevant_documents(question)[:max_documents]

    # Combina o conteúdo dos documentos recuperados
    context = "\n".join([doc.page_content for doc in docs])

    # Trunca o contexto se exceder max_context_tokens
    context_tokens = tokenizer.tokenize(context)
    if len(context_tokens) > max_context_tokens:
        context_tokens = context_tokens[:max_context_tokens]
        context = tokenizer.convert_tokens_to_string(context_tokens)

    sources = [doc.metadata for doc in docs]
    return context, sources

def remove_repetitions(text):
    """
    Remove frases repetidas consecutivas do texto.
    """
    # Padrão de regex para encontrar frases repetidas consecutivamente
    pattern = re.compile(r'(.+?)(?:\s+\1)+', re.IGNORECASE)
    return pattern.sub(r'\1', text)

def generate_response(question):
    """
    Gera uma resposta à pergunta do usuário usando a integração RAG.
    """
    # Recupera contexto relevante e fontes
    context, sources = get_relevant_context(question)

    prompt = f"Question: {question}\nContext: {context}\nAnswer:"

    # Gera a resposta usando o pipeline
    response = nlp(
        prompt,
        max_new_tokens=max_new_tokens, # Define o número de novos tokens a serem gerados
        temperature=0.3,            # Ajuste para criatividade
        top_k=50,                   # Controla diversidade através da amostragem top-k
        top_p=0.9,                  # Controla diversidade via amostragem nucleus (valor corrigido)
        repetition_penalty=2.0,     # Penaliza frases repetidas
        no_repeat_ngram_size=5,     # Previne repetição de n-grams
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = response[0]['generated_text']
    # Remove o prompt do texto gerado para obter a resposta
    answer = generated_text[len(prompt):].strip()
    # Limpa a resposta removendo repetições
    answer = remove_repetitions(answer)
    formatted_sources = "\n".join([f"- UID: {source['uid']}, Title: {source['title']}" for source in sources])

    return f"\n**Answer:**\n{answer}\n\n**Sources:**\n{formatted_sources}"

In [30]:
docs = retriever.get_relevant_documents("Tell me something about Asian food")[:10]
docs

[Document(metadata={'uid': 'B006S2SJKM', 'title': 'HuangFeiHong Spicy Snack Peanuts 3.8 oz 110G 8 bags Pack of 8'}, page_content='HuangFeiHong Spicy Snack Peanuts 8 3.8 oz 110G 8 bags'),
 Document(metadata={'uid': '0786864753', 'title': "Sam Choy's Polynesian Kitchen More Than 150 Authentic Dishes from One of the World's Most Delicious and Overlooked Cuisines"}, page_content="Hawaiian chef and restaurant owner Choy Sam Choy's Island Flavors brings to the table his island tales and food with this delightful new volume. Filled with anecdotes and photographs of his visits and the food memories associated with each voyage, he travels as far south as New Zealand, taking in along the way such farreaching islands as Samoa, Tonga and Tahiti. Starting with a very full section covering the ingredients used throughout the book, Choy discusses the various culinary influences from the Chinese in Fiji and Samoa and the French in the Marquesas, to the Indian British impact throughout the regions. Whe

# Geração de Respostas
- Configure o modelo treinado para receber perguntas dos usuários.
Quando uma pergunta for recebida, utilize a integração RAG para recuperar informações relevantes do dataset AmazonTitles-1.3MM.

In [47]:
user_question = "What features make wireless headphones ideal for sports?"
response = generate_response(
    user_question
)
print(response)


**Answer:**
Your smartphone or tablet will receive voice commands from your favorite song player using this device. You'll also receive text messages via Skype while playingamesuch as Guitar Hero 2, Game Boy Advance ClassicD Player game play, Rock Band 4Droid VHS video streamerPlayStation Portable MP3R Video Streaming HDTVMVTT, and other popular entertainment platforms.This product may not work properly due back issues caused by software update whichas been disableduring installation.Please notethat any errors areported when installing our products through our website'support page

**Sources:**
- UID: B00DQB0JVK, Title: Puma Social Earbud Wmic Pink
- UID: B00B4DFNJU, Title: Califone 2800 Listening First Headphone 3 Packs Blue, Yellow, Red
- UID: B004QOA93E, Title: SONY DREX12IP InEar Stereo Headphones with Mic and Remote Black
- UID: B009AWT01Y, Title: JBL Flip Wireless Bluetooth Speaker White
- UID: B001SUZNGW, Title: Shark Motorcycle Audios Shkmbt88i Snowmobile Bluetooth Multi Inter

# Exclusão
Antes de sair, não esqueça de deletar os arquivos criados executando a célula abaixo.

In [154]:
import shutil
import os

if os.path.exists(FOLDER_PATH):
    try:
        shutil.rmtree(FOLDER_PATH)
        print(f'Diretório "{FOLDER_PATH}" e todo o seu conteúdo foram excluídos com sucesso.')
    except Exception as e:
        print(f'Erro ao tentar excluir o diretório "{FOLDER_PATH}": {e}')
else:
    print(f'O diretório "{FOLDER_PATH}" não existe.')


Diretório "/content/drive/MyDrive/TheAmazonTitles" e todo o seu conteúdo foram excluídos com sucesso.
